# Train a Keras Sequential Model
## Using experiment
This notebook shows how to train a Keras Sequential model on SageMaker, and use SageMaker Experiment Management Python SDK to organize, track, compare, and evaluate your machine learning (ML) model training experiments.

You can track artifacts for experiments, including data sets, algorithms, hyper-parameters, and metrics. Experiments executed on SageMaker such as SageMaker Autopilot jobs and training jobs will be automatically tracked. You can also track artifacts for additional steps within an ML workflow that come before/after model training e.g. data pre-processing or post-training model evaluation.

The APIs also let you search and browse your current and past experiments, compare experiments, and identify best performing models.

The model used for this notebook is a simple deep CNN that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py). The experiment will be organized as follow:

1. Download and prepare the cifar10 dataset.
2. Train a Convolutional Neural Network (CNN) Model. Tune the hyper parameter that configures the number of hidden channels in the model. Track the parameter configurations and resulting model accuracy using SageMaker Experiments Python SDK.
3. Finally use the search and analytics capabilities of Python SDK to search, compare and evaluate the performance of all model versions generated from model tuning in Step 2.
4. We will also see an example of tracing the complete linage of a model version i.e. the collection of all the data pre-processing and training configurations and inputs that went into creating that model version.







## The dataset
The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is one of the most popular machine learning datasets. It consists of 60,000 32x32 images belonging to 10 different classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

In this tutorial, we will train a deep CNN to recognize these images.

We'll compare trainig with file mode, pipe mode datasets and distributed training with Horovod

## Set up the environment

In [47]:
import time
import os
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.analytics import ExperimentAnalytics
boto3_session = boto3.Session()
sm_client = boto3_session.client('sagemaker')
sm_session = sagemaker.Session(boto_session=boto3_session, sagemaker_client=sm_client)
role = get_execution_role()

In [8]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

  Using cached sagemaker_experiments-0.1.13-py3-none-any.whl (35 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [9]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

## Download the CIFAR-10 dataset
Downloading the test and training data takes around 5 minutes.

In [2]:
#!pip install wget
# import wget # for TF2
!python generate_cifar10_tfrecords_v1.x.py --data-dir data/


Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.
Generating data//train/train.tfrecords


Generating data//validation/validation.tfrecords
Generating data//eval/eval.tfrecords
Done!


## Run on SageMaker cloud

### Uploading the data to s3

In [3]:
dataset_location = sm_session.upload_data(path='data', key_prefix='data/DEMO-cifar10-tf')
display(dataset_location)

's3://sagemaker-us-east-1-079329190341/data/DEMO-cifar10-tf'

Now lets track the parameters from the data pre-processing step.

In [24]:
with Tracker.create(display_name="Preprocessing", sagemaker_boto_client=sm_client) as tracker:
    tracker.log_parameters({
        "datatype": 'tfrecords',
        "image_size": 32,
    })
    # we can log the s3 uri to the dataset we just uploaded
    tracker.log_input(name="cifar10-dataset", media_type="s3/uri", value=dataset_location)

### Configuring metrics from the job logs
SageMaker can get training metrics directly from the logs and send them to CloudWatch metrics.

In [4]:
keras_metric_definition = [
    {'Name': 'train:loss', 'Regex': '.*loss: ([0-9\\.]+) - acc: [0-9\\.]+.*'},
    {'Name': 'train:accuracy', 'Regex': '.*loss: [0-9\\.]+ - acc: ([0-9\\.]+).*'},
    {'Name': 'validation:accuracy', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: ([0-9\\.]+).*'},
    {'Name': 'validation:loss', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_acc: [0-9\\.]+.*'},
    {'Name': 'sec/steps', 'Regex': '.* - \d+s (\d+)[mu]s/step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: [0-9\\.]+'}
]

### Train image classification based on the cifar10 dataset

In [28]:
cifar10_experiment = Experiment.create(
    experiment_name=f"cifar10-image-classification-{int(time.time())}", 
    description="Classification of images", 
    sagemaker_boto_client=sm_client)
print(cifar10_experiment)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fb43f10e748>,experiment_name='cifar10-image-classification-1590035896',description='Classification of images',experiment_arn='arn:aws:sagemaker:us-east-1:079329190341:experiment/cifar10-image-classification-1590035896',response_metadata={'RequestId': '21d250dc-ce71-4a5d-bf6c-55d3007e8b17', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '21d250dc-ce71-4a5d-bf6c-55d3007e8b17', 'content-type': 'application/x-amz-json-1.1', 'content-length': '111', 'date': 'Thu, 21 May 2020 04:38:15 GMT'}, 'RetryAttempts': 0})


In [29]:
preprocessing_trial_component = tracker.trial_component

In [38]:
source_dir = os.path.join(os.getcwd(), 'source_dir')

In [39]:
num_epochs_trial_name_map = {}

In [ ]:
for i, num_epochs in enumerate([1]):
    # create trial
    trial_name = f"cifar10-training-job-{num_epochs}-epochss-{int(time.time())}"
    cifar10_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=cifar10_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    num_epochs_trial_name_map[num_epochs] = trial_name
    
    # associate the proprocessing trial component with the current trial
    cifar10_trial.add_trial_component(preprocessing_trial_component)


    estimator = TensorFlow(base_job_name='cifar10-tf',
                           entry_point='cifar10_keras_main.py',
                           source_dir=source_dir,
                           role=role,
                           framework_version='1.12.0',
                           py_version='py3',
                           hyperparameters={'epochs': num_epochs, 'batch-size' : 256, 'optimizer' : 'sgd'},
                           train_instance_count=1, train_instance_type='ml.p3.2xlarge',
                           metric_definitions=keras_metric_definition)
    
    cifar10_training_job_name = "cifar-training-job-{}".format(int(time.time()))
    remote_inputs = {'train' : dataset_location+'/train', 'validation' : dataset_location+'/validation', 'eval' : dataset_location+'/eval'}
    estimator.fit(remote_inputs, job_name=cifar10_training_job_name,
        experiment_config={
            "TrialName": cifar10_trial.trial_name,
            "TrialComponentDisplayName": "Training",
        },
        wait=True,)
    # give it a while before dispatching the next training job
    time.sleep(2)

INFO:sagemaker:Creating training-job with name: cifar-training-job-1590072815


2020-05-21 14:53:35 Starting - Starting the training job..

In [42]:
num_epochs_trial_name_map

{10: 'cifar10-training-job-10-epochss-1590037315',
 1: 'cifar10-training-job-1-epochss-1590037352',
 2: 'cifar10-training-job-2-epochss-1590037698',
 3: 'cifar10-training-job-3-epochss-1590038045'}

In [43]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

In [48]:
trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sm_session, 
    experiment_name=cifar10_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.test:accuracy.max",
    sort_order="Descending",
    metric_names=['test:accuracy'],
    parameter_names=['hidden_channels', 'epochs', 'dropout', 'optimizer']
)

In [49]:
trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,epochs
0,cifar-training-job-1590037352-aws-training-job,Training,arn:aws:sagemaker:us-east-1:079329190341:train...,10.0
1,cifar-training-job-1590038045-aws-training-job,Training,arn:aws:sagemaker:us-east-1:079329190341:train...,10.0
2,cifar-training-job-1590037315-aws-training-job,Training,arn:aws:sagemaker:us-east-1:079329190341:train...,10.0
3,cifar-training-job-1590037698-aws-training-job,Training,arn:aws:sagemaker:us-east-1:079329190341:train...,10.0


In [52]:
lineage_table = ExperimentAnalytics(
    sagemaker_session=sm_session, 
    search_expression={
        "Filters":[{
            "Name": "Parents.TrialName",
            "Operator": "Equals",
            "Value": num_epochs_trial_name_map[1]
        }]
    },
    sort_by="CreationTime",
    sort_order="Ascending",
)

In [53]:
lineage_table.dataframe()

,TrialComponentName,DisplayName,datatype,image_size,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,batch-size,...,train:accuracy - Avg,train:accuracy - StdDev,train:accuracy - Last,train:accuracy - Count,train:loss - Min,train:loss - Max,train:loss - Avg,train:loss - StdDev,train:loss - Last,train:loss - Count
0,TrialComponent-2020-05-21-043247-krju,Preprocessing,tfrecords,32.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cifar-training-job-1590037352-aws-training-job,Training,NaN,NaN,arn:aws:sagemaker:us-east-1:079329190341:train...,520713654638.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.p3.2xlarge,30.0,256.0,...,0.447743,0.130066,0.5893,379.0,1.1105,4.2163,1.549224,0.449781,1.1466,380.0
